# Reporte Práctica 2, Parte 2
*Lau, Santi, Rafa y Sebas*
**Optimización Avanzada**
***

## Introducción

 En esta última parte de las prácticas desarrolladas durante el curso se reimplementan los métodos que se han desarrollado anteriormente, haciendo el código más eficiente mediante el uso de *multiprocessing*.

## Definición del Problema

 En este ocasión trabajaremos nuevamente con el dataset que contiene las ciudades de China, `ch71009.tsp` que se encuentra en el directorio `datasets` de este repositorio$^{[2]}$. Tomaremos una muestra aleatoria de $100$ ciudades y $1000$ hormigas, de esta manera podremos comprobar que se ha hecho una mejora en el tiempo de cómputo en el algoritmo.

## Perfilamiento

 La reimplementación desarrollada se hizo sobre la clase `colony` a una versión que utiliza multiprocesamiento, que se apoya de la librería `multiprocessing` que revisamos en el libro del curso $^{[1]}$ sobre el cómputo en paralelo.

 De los métodos que se desarrollaron anteriormente, notemos que el método `solve_tsp` es el que toma más tiempo durante su ejecución, por lo que en esta práctica se implementó la siguiente solución:
 
 - Se envía un número determinado de hormigas a un *pool* de workers, donde para cada iteración, las hormigas van a recorrer el grafo para buscar una solución.
 - En cada iteración se actualiza el número de feromonas del grafo según los recorridos determinados por cada una de las hormigas del punto anterior.

In [52]:
!pip install memory_profiler

     |████████████████████████████████| 291 kB 44.6 MB/s eta 0:00:01
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=36588 sha256=62c592798a6aeabb66577a0e387675d5b8b5f37bcc815466badcfea82e2b8956
  Stored in directory: /root/.cache/pip/wheels/8c/47/b0/6aa7f5774be599d4f5256b58061f8264dd0ec24bb9de56f568
Successfully built memory-profiler


In [53]:
import timeit
import cProfile
import pstats
import memory_profiler
from memory_profiler import memory_usage
import line_profiler

 Primero echemos un vistazo a las características de la máquina en la que se ejecuta este reporte:

In [42]:
!lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              4
On-line CPU(s) list: 0-3
Thread(s) per core:  2
Core(s) per socket:  2
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8259CL CPU @ 2.50GHz
Stepping:            7
CPU MHz:             3099.932
BogoMIPS:            4999.99
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            36608K
NUMA node0 CPU(s):   0-3
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand h

Veremos el perfilamiento de tiempos de ejecución:

## Implementación con `ant_colony`

In [20]:
#!pip install "git+https://github.com/optimizacion-2-2021-1-gh-classroom/practica-1-segunda-parte-ltejadal.git#egg=ant-colony&subdirectory=src" &> /dev/null

In [26]:
# librerias
import ant_colony as ac
import time

In [41]:
path_china = 'datasets/ch71009.tsp'
G = ac.read_coord_data(path_china, n_cities=100, seed=1999)

Problem with 71009 cities. Selected 100.


In [43]:
n_ants = 1000

Vamos a medir el tiempo de ejecución con la clase anterior y compararla contra la nueva.

In [44]:
colony_old = ac.colony(G, init_node=0,  n_ants=n_ants)

In [45]:
start_time = time.time()
colony_old.solve_tsp()
end_time = time.time()

In [46]:
secs = end_time-start_time
print("La solucion sin pool tomó", secs, "segundos." )
print(f"Distancia {colony_old.best_dist} kms.")

La solucion sin pool tomó 2699.3665103912354 segundos.
Distancia 1119.0648040721273 kms.


Ahora veremos con `cProfile` las partes del método que tardan más en su ejecución:

In [56]:
cprof = cProfile.Profile()
cprof.enable()
res =  colony_old.solve_tsp()
cprof.disable()
cprof.print_stats(sort='cumtime')

         3203921846 function calls (3203921746 primitive calls) in 3636.850 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000 3636.850 1818.425 interactiveshell.py:3288(run_code)
        2    0.000    0.000 3636.850 1818.425 {built-in method builtins.exec}
        1    0.000    0.000 3636.850 3636.850 <ipython-input-56-fdb53b4ac090>:3(<module>)
        1    0.111    0.111 3636.850 3636.850 aco_tsp_oo.py:278(solve_tsp)
      100  115.680    1.157 3634.949   36.349 aco_tsp_oo.py:247(_colony_run)
   100000  245.426    0.002 3519.218    0.035 aco_tsp_oo.py:317(walk_over_graph)
   100000  693.978    0.007 3031.069    0.030 utils.py:70(create_dic_dist_from_graph)
1010100000 1205.066    0.000 1475.698    0.000 defmatrix.py:189(__getitem__)
   100000    1.966    0.000  860.889    0.009 convert_matrix.py:442(to_numpy_matrix)
   100000  615.408    0.006  853.594    0.009 convert_matrix.py:1093(to_numpy_arr

In [57]:
cprof.dump_stats("solve_tsp_stats")

In [58]:
p_solve_tsp_stats = pstats.Stats("solve_tsp_stats")
print(p_solve_tsp_stats.sort_stats("cumulative").print_stats(10))

Sun May  9 17:10:50 2021    solve_tsp_stats

         3203921846 function calls (3203921746 primitive calls) in 3636.850 seconds

   Ordered by: cumulative time
   List reduced from 69 to 10 due to restriction <10>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
        2    0.000    0.000 3636.850 1818.425 /usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3288(run_code)
        2    0.000    0.000 3636.850 1818.425 {built-in method builtins.exec}
        1    0.000    0.000 3636.850 3636.850 <ipython-input-56-fdb53b4ac090>:3(<module>)
        1    0.111    0.111 3636.850 3636.850 /usr/local/lib/python3.6/dist-packages/ant_colony/aco_tsp_oo.py:278(solve_tsp)
      100  115.680    1.157 3634.949   36.349 /usr/local/lib/python3.6/dist-packages/ant_colony/aco_tsp_oo.py:247(_colony_run)
   100000  245.426    0.002 3519.218    0.035 /usr/local/lib/python3.6/dist-packages/ant_colony/aco_tsp_oo.py:317(walk_over_graph)
   100000  693.978    0.007 

 Las que más se tardan en su ejecución son:
 
 - Métodos de `ant_colony`:
     - `walk_over_graph`
     - `create_dict_dist_from_graph`
 - Método de `numpy`:    
     - `get_item`
 - Método de `networkx`:
     - `convert_matrix`
     
 Nos concentramos en perfilar los métodos de `ant_colony` que más se tardan, ya que nuestro objetivo es mejorar la implementación del algoritmo.

## Implementación con `colony_mutiw`

In [47]:
colony_mw = ac.colony_multiw(G, init_node=0,  n_ants= n_ants, n_workers=4)

In [48]:
start_time = time.time()
colony_mw.solve_tsp()
end_time = time.time()

In [49]:
secs = end_time-start_time
print("La solucion con pool de workers tomó", secs, "segundos." )
print(f"Distancia {colony_mw.best_dist} kms.")

La solucion con pool de workers tomó 452.3769097328186 segundos.
Distancia 1119.0648040721273 kms.


El tiempo de ejecución se ha reducido casi 6 veces respecto a la clase que habíamos definido anteriormente.

## Conclusión

 Notemos que en ambos casos la distancia se mantiene fija, pero el tiempo de ejecución se ha reducido sustancialmente. Con esta nueva implementación en la que se utiliza cómputo en paralelo el tiempo de cómputo se redujo aproximadamente 6 veces, consiguiendo el objetivo de optimización del algoritmo.

## Referencias

1. [Cómputo en paralelo usando CPUs en un sistema de memoria compartida (SMC)](https://itam-ds.github.io/analisis-numerico-computo-cientifico/V.optimizacion_de_codigo/5.4/Computo_en_paralelo_usando_CPUS_en_SMC.html#multiprocessing) (2021) Erick Palacios
2. [National Travelling Salesman Problems](https://www.math.uwaterloo.ca/tsp/world/countries.html) (2017) University of Waterloo